In [ ]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla

--2024-01-16 10:15:22--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2024-01-16 10:15:24--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdf50ad98e45d5dba353581be8e.dl.dropboxusercontent.com/zip_download_get/Bu3TEnp_oDDFVxBNb6SbAgOUf_YK-kZjdQ4abzWENx_LkZDuNqcR-WAriuOCrhi6OUeGAPOSfmD-RVIBSVVbAcJksBH_I3_zyTEbK_skcbC9GA# [following]
--2024-01-16 10:15:24--  https://ucdf50ad98e45d5dba353581be8e.dl.dropboxusercontent.com/zip_download_get/Bu3TEnp_oDDFVxBNb6SbAgOUf_YK-kZjdQ4abzWENx_LkZDuNqcR-WAriuOCrhi6OUeGAPOSfmD-RVIBSVVbAcJksBH_I3

In [ ]:

!unzip /content/AAAw1_JIzpuVvwteJCma0xMla

Archive:  /content/AAAw1_JIzpuVvwteJCma0xMla
mapname:  conversion of  failed
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: train.csv               


In [ ]:
import re
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# New Section
**Data preprocessing:**
  Load Data

  Text Cleaning
  Tokenization
  Padding
** Glove Embeddings**
    Download Glove embeddings
    Create Embedding matrix
 ** Model Building with TensorFlow2**
    Define model architecture
    Embedding layer
    Add hidden layers
    output layer
    compile the model
   ** Training the model**
   prepare training and validation sets
   model training
 **  Model Evaluation and prediction**
   Evaluate model performance
   prediction


In [ ]:
@np.vectorize
def clean_text(text):
  text=text.lower()
  tokens=word_tokenize(text)
  stop_words=set(stopwords.words('english'))
  filtered_tokens = [token for token in tokens if token not in stop_words]
  text=''.join(filtered_tokens)
  return text

In [ ]:

import pandas as pd


data=pd.read_csv('/content/train.csv')

In [ ]:
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
import numpy as np
x,y=data.question_text.to_numpy()[:,np.newaxis],data.target.to_numpy()

In [ ]:
x.shape

(1306122, 1)

In [ ]:
y.shape

(1306122,)

In [ ]:
x=clean_text(x)


In [ ]:
from tensorflow.keras.preprocessing import sequence
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000)

x= x.flatten()

tokenizer.fit_on_texts(x)
sequences=tokenizer.texts_to_sequences(x)
padded_sequences=sequence.pad_sequences(sequences)


In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip /content/glove.42B.300d.zip
!rm /content/glove.42B.300d.zip

--2024-01-16 09:38:01--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2024-01-16 09:38:02--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2024-01-16 09:38:02--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
glove_file=r'/content/glove.42B.300d.txt'
embeddings_index={}
with open(glove_file,'r',encoding='utf-8') as f:
   for line in f:
       values=line.split()
       word=values[0]
       vector=np.asarray(values[1:],dtype='float32')
       embeddings_index[word]=vector
print(f'loaded{len(embeddings_index)}word vectors')

loaded1917494word vectors


In [ ]:
#now creating embedding matrix
word_index=tokenizer.word_index
vocab_size=len(word_index)+1

embedding_dim=300
embedding_matrix=np.zeros((vocab_size,embedding_dim))
for word,i in word_index.items():
  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
      embedding_matrix[i]==embedding_vector


In [ ]:
vocab_size=len(tokenizer.word_index)+1

embedding_dim=300
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

model=Sequential([Embedding(vocab_size,embedding_dim,input_length=98),
                  LSTM(128,return_sequences=True),
                  LSTM(64),
                  Dense(1,activation='sigmoid')
                  ])
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable=False

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val=train_test_split(padded_sequences,y,test_size=0.2,random_state=42)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit(x_train,y_train,batch_size=1024,epochs=2,validation_data=(x_val,y_val))

Epoch 1/2
1021/1021 [==============================] - 42s 36ms/step - loss: 0.2364 - accuracy: 0.9378 - val_loss: 0.2289 - val_accuracy: 0.9393
Epoch 2/2
1021/1021 [==============================] - 36s 36ms/step - loss: 0.2330 - accuracy: 0.9378 - val_loss: 0.2290 - val_accuracy: 0.9393


In [ ]:
padded_sequences.shape

(1306122, 98)

In [ ]:
type(padded_sequences)

numpy.ndarray